## Try using aips and difmap coding to simulate Monte Carlo for antenna gain errors

### The operation here:
1. Generate gain facotors for each antenna at each time step(in python)
2. load and correct gain factors (aips, parseltongue)
3. export corrected data to difmap and run modelfit in difmap (python scripting)
4. Save results and analyze in python


In [6]:
import pexpect
import os
import sys
import re
import time

In [1]:
def init_difmap(timeout=8000000):
    """初始化 difmap 会话并返回 difmap 对象和日志文件名"""
    difmap = pexpect.spawn('difmap')
    difmap.waitnoecho
    difmap.expect('0>')
    p = re.compile(r'(difmap.log_?\d*)')
    logfile = p.findall(difmap.before.decode())[0]
    difmap.timeout = timeout
    return difmap, logfile

def setup_mapsize(difmap, freq):
    """根据频率设置地图大小"""
    if freq <= 3:
        difmap.sendline('mapsize 2048,0.4')
    elif 3.1 <= freq <= 10:
        difmap.sendline('mapsize 2048,0.2')
    elif freq >= 10.1:
        difmap.sendline('mapsize 2048,0.1')
    difmap.expect('0>')

def prepare_observation(difmap, filename, freq):
    """准备观测：加载文件，选择偏振，设置地图大小和 UV 权重"""
    difmap.sendline('obs %s' % filename)
    difmap.expect('0>')
    difmap.sendline('select i')
    difmap.expect('0>')
    setup_mapsize(difmap, freq)
    difmap.sendline('uvw 0,-2')
    difmap.expect('0>')

In [7]:
difmap,_=init_difmap()

In [ ]:
def get_model_parm(difmap, prompt=r"0>"):
    """
    在 difmap 里执行一条命令，并返回从命令回显到下一个提示符之间的全部文本输出。
    """
    cmd = 'modelfit 0'
    difmap.sendline(cmd)
    difmap.expect(prompt)
    out = difmap.before or ""
    out = out.replace("\r", "")

    # difmap 通常会回显你输入的命令，把第一行是命令的情况去掉
    lines = out.splitlines()
    if lines and lines[0].strip() == cmd.strip():
        lines = lines[1:]
    return "\n".join(lines).strip()

In [ ]:
text = get_model_parm(difmap)
print(text)

### 1. Generate gain factors for each antenna at each time step (in Python)

In [ ]:
def gen_antenna_gains(
    nants: int,
    gain_range: float = 0.1,
    dist: str = "uniform",
    seed: int | None = None,
) -> np.ndarray:
    """
    返回每根天线的幅度增益因子 g_i(real>0)。

    gain_range=0.1 -> uniform: [0.9, 1.1]
                      gaussian: mean=1, sigma=gain_range/2, 严格限定在 [0.9, 1.1]
    dist: "uniform" 或 "gaussian"   
    """
    rng = np.random.default_rng(seed)
    lo, hi = 1 - gain_range, 1 + gain_range

    if dist.lower() == "uniform":
        g = rng.uniform(lo, hi, size=nants)
    elif dist.lower() == "gaussian":
        sigma = gain_range / 2
        g = 1 + rng.normal(loc=0.0, scale=sigma, size=nants)
        g = np.clip(g, a_min=lo, a_max=hi)  # 严格限定在 [lo, hi] 范围内
    else:
        raise ValueError("dist must be 'uniform' or 'gaussian'")

    return g

In [ ]:
t = []
for i in range (10):
    g = gen_antenna_gains(8, gain_range=0.1, dist="gaussian", seed=None)
    t.append(g)
print(t)

[array([0.9763676 , 1.0164002 , 0.9883903 , 0.94875331, 1.01585807,
       1.0263361 , 1.02301885, 1.03927169]), array([0.97558872, 0.9512307 , 1.04055993, 0.9414121 , 1.04044525,
       0.97632577, 1.01796045, 1.00207396]), array([0.92068308, 0.92787154, 0.95531135, 0.9833696 , 0.95107106,
       1.01226005, 1.1       , 1.05468739]), array([0.92119516, 1.05095443, 1.04874986, 1.05093515, 0.99882777,
       0.97969162, 1.01892171, 0.95414144]), array([0.91318809, 1.00709351, 1.0052809 , 1.02656733, 0.96942443,
       1.04229469, 1.06214136, 1.01681327]), array([0.95475536, 0.96679453, 0.96954847, 1.01090309, 1.02313562,
       0.99665033, 1.09189165, 0.97684882]), array([0.97034512, 0.93203715, 1.1       , 0.98672983, 0.93517154,
       0.95824501, 0.95098042, 1.01499824]), array([0.97730366, 0.95187742, 1.00770207, 0.97729677, 0.96503017,
       0.98371663, 0.99481543, 1.05572956]), array([0.9       , 1.00218636, 1.01846571, 0.9546945 , 1.04148847,
       0.95918359, 1.03047067, 1.032

### 2. correct gain factors in aips

In [12]:
import sys
import os
# 获取notebook所在目录的父目录
sys.path.append('../vlbi-pipeline/')
from utils import gcal_apply

ModuleNotFoundError: No module named 'AIPS'

In [ ]:
tar_org_uvf = AIPSUVData(target_name,'uvf',1,1)
loadfr(filepath, filename, outname, outclass, outdisk, antname) # from run_tasks
tar_1_uvf = AIPSUVData(target_name+'_1','uvf',1,1)
copy_uvdata(indata, outname, outclass)
gcal_apply(indata,matx,cluse,pol)

run_split2(indata, source, gainuse, outclass, doband, bpver, flagver,av_chan, split_seq)